# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,7.85,81,75,2.06,CA,1690392031
1,1,hoemul-li,41.4339,129.6700,22.47,88,100,1.60,KP,1690392037
2,2,palikir - national government center,6.9248,158.1611,27.00,82,100,6.62,FM,1690392146
3,3,porto novo,6.4965,2.6036,26.89,77,67,4.52,BJ,1690392154
4,4,lebu,-37.6167,-73.6500,10.04,76,100,2.45,CL,1690392156


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_city_data_df = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = 'Humidity',
    frame_width = 700,
    frame_height = 600,
    alpha = 0.5,
    xlabel = "Longitude (Degrees)",
    ylabel = "Latitude (Degrees)"
)

# Save map
holoviews.renderer('bokeh').save(map_city_data_df, './output_data/map_all_cities_in_df', fmt='png')

# Display the map
map_city_data_df

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
limit = 10
ideal_df = city_data_df[(city_data_df['Max Temp'] >= 20) 
                        & (city_data_df['Max Temp'] <= 27) 
                        & (city_data_df['Wind Speed'] < 4.5)
                        & (city_data_df['Cloudiness'] == 0)].head(limit)

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,stanley,54.8680,-1.6985,20.01,58,0,3.09,GB,1690392161
102,102,creswell,43.9179,-123.0245,21.31,68,0,1.73,US,1690392176
159,159,altamont,42.2068,-121.7372,23.83,56,0,0.89,US,1690392024
317,317,dudinka,69.4058,86.1778,20.90,70,0,3.73,RU,1690392236
324,324,berlin,52.5244,13.4105,20.65,57,0,4.47,DE,1690392237
368,368,sinop,41.7711,34.8709,23.34,50,0,3.29,TR,1690392246
372,372,rakops,-21.0167,24.3333,20.77,31,0,4.05,BW,1690392247
485,485,mammoth lakes,37.6485,-118.9721,23.52,23,0,2.06,US,1690392275
502,502,saint-francois,46.4154,3.9054,24.69,38,0,1.54,FR,1690392279


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,stanley,GB,54.8680,-1.6985,58,
102,creswell,US,43.9179,-123.0245,68,
159,altamont,US,42.2068,-121.7372,56,
317,dudinka,RU,69.4058,86.1778,70,
324,berlin,DE,52.5244,13.4105,57,
368,sinop,TR,41.7711,34.8709,50,
372,rakops,BW,-21.0167,24.3333,31,
485,mammoth lakes,US,37.6485,-118.9721,23,
502,saint-francois,FR,46.4154,3.9054,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
stanley - nearest hotel: Hotel 52
creswell - nearest hotel: Comfort Inn & Suites
altamont - nearest hotel: Quality Inn
dudinka - nearest hotel: Виктория
berlin - nearest hotel: Lux 11
sinop - nearest hotel: No hotel found
rakops - nearest hotel: Rakops River Lodge
mammoth lakes - nearest hotel: Travelodge by Wyndham Mammoth Lakes
saint-francois - nearest hotel: Chez Lily


,City,Country,Lat,Lng,Humidity,Hotel Name
11,stanley,GB,54.8680,-1.6985,58,Hotel 52
102,creswell,US,43.9179,-123.0245,68,Comfort Inn & Suites
159,altamont,US,42.2068,-121.7372,56,Quality Inn
317,dudinka,RU,69.4058,86.1778,70,Виктория
324,berlin,DE,52.5244,13.4105,57,Lux 11
368,sinop,TR,41.7711,34.8709,50,No hotel found
372,rakops,BW,-21.0167,24.3333,31,Rakops River Lodge
485,mammoth lakes,US,37.6485,-118.9721,23,Travelodge by Wyndham Mammoth Lakes
502,saint-francois,FR,46.4154,3.9054,38,Chez Lily


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = 'Humidity',
    frame_width = 700,
    frame_height = 600,
    alpha = 0.5,
    xlabel = "Longitude (Degrees)",
    ylabel = "Latitude (Degrees)",
    hover_cols = ['Hotel Name','Country']
)

# Save map
holoviews.renderer('bokeh').save(hotels_plot, './output_data/map_hotels_df', fmt='png')

# Display the map
hotels_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)